In [1]:
import sys
sys.path.append('/kaggle/input/rus-model/Emotion Detection Kaggle')
import json, torch
from LoadData import load_dataset
from Model import Model
from EmotionDataset import EmotionDataset
from train import train
from plot import plot
from predict import predict
from sklearn.model_selection import KFold
import numpy as np
from analyze import analyze

In [2]:
with open("/kaggle/input/rus-model/Emotion Detection Kaggle/config.json", "r") as file:
    config = json.load(file)
batch_size = config["batch_size"]
train_path = config["train_path"]
test_path = config["test_path"]
learning_rate = config["learning_rate"]
base_model = config["base_model"]
num_labels = config["num_labels"]
problem_type = config["problem_type"]
labels = config["LABELS"]
num_epoch = config["num_epoch"]
eps = config["eps"]
th = config["th"]
random_state = config["random_state"]
n_splits = config["n_splits"]
Min_loss = config["Min_loss"]
num_folds = config["num_fold"]

In [3]:
X, y, X_test, test_df = load_dataset(train_path, test_path)
kfold = KFold(n_splits=n_splits, shuffle=True, random_state=random_state)

In [4]:
from sklearn.metrics import f1_score
def find_threshold(model, valid_loader):
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu') 
    for th in np.arange(0.2, 0.7, 0.01):
        f1score_sum = 0
        for batch in valid_loader:
            batch = {k: v.to(device) for k, v in batch.items() if k != "text"}
            with torch.no_grad():
                outputs = model(**batch)
            logits = outputs.logits
            predictions = np.array(torch.sigmoid(logits).tolist())
            predictions = np.where(predictions > th, 1, 0)
            label = batch["labels"].cpu().numpy()
            f1score_sum += f1_score(label, predictions, average='macro', zero_division=1)
            
        f1score_avg = (f1score_sum / len(valid_loader)).item()
        print(f"threshold = {th:.3f}   -   f1-score = {f1score_avg:.5f}")

In [5]:
best_models = [] 
fold_valid_losses = [] 
wrong_predcit = []

for fold, (train_idx, val_idx) in enumerate(kfold.split(X)):
    X_train, X_valid = X[train_idx].tolist(), X[val_idx].tolist()
    y_train, y_valid = y[train_idx].tolist(), y[val_idx].tolist()
    model, tokenizer = Model(base_model, num_labels, problem_type, labels).build()
    train_loader, valid_loader, test_loader = EmotionDataset(tokenizer, batch_size).build_dataset(X_train, y_train, X_valid, y_valid, X_test)

    optimizer = torch.optim.AdamW(model.parameters(), lr = learning_rate, betas = (0.9, 0.98), eps = eps, weight_decay=0.01)
    model, train_losses, valid_losses, train_f1scores, valid_f1scores = train(num_epoch, model, train_loader, valid_loader, optimizer, th, Min_loss)
    wrong_predcit.append(analyze(model, valid_loader, th))
    #find_threshold(model, valid_loader)
    #plot(train_losses, valid_losses, "loss", "Loss training and valid curve", num_epoch)
    #plot(train_f1scores, valid_f1scores, "F1-score", "F1-score training and valid", num_epoch)
    
    Min_valid_loss = valid_losses[-1]
    fold_valid_losses.append(Min_valid_loss)

    if Min_valid_loss < Min_loss:
        model_save_path = f'model_fold_{fold + 1}_loss_{Min_valid_loss:.4f}.pt'
        torch.save({
            'fold': fold + 1,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'valid_loss': Min_valid_loss,
            'train_losses': train_losses,
            'valid_losses': valid_losses,
            'train_f1scores': train_f1scores,
            'valid_f1scores': valid_f1scores
        }, model_save_path)
        best_models.append({
            'fold': fold + 1,
            'model': model,
            'loss': Min_valid_loss,
            'path': model_save_path
        })
        print(f"Saved model for fold {fold + 1} with validation loss: {Min_valid_loss:.4f}\n\n\n")
    if fold == num_folds-1:
        break

tokenizer_config.json:   0%|          | 0.00/373 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/634 [00:00<?, ?B/s]

...

Epoch 1 - Loss: 0.348 - F1-score: 0.319 - val_loss: 0.228 - valid-F1-score: 0.580
Epoch 2 - Loss: 0.153 - F1-score: 0.801 - val_loss: 0.163 - valid-F1-score: 0.780
Epoch 3 - Loss: 0.100 - F1-score: 0.871 - val_loss: 0.166 - valid-F1-score: 0.798
Epoch 4 - Loss: 0.075 - F1-score: 0.910 - val_loss: 0.154 - valid-F1-score: 0.827


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-large and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.334 - F1-score: 0.376 - val_loss: 0.169 - valid-F1-score: 0.823
Epoch 2 - Loss: 0.150 - F1-score: 0.808 - val_loss: 0.130 - valid-F1-score: 0.816
Epoch 3 - Loss: 0.108 - F1-score: 0.869 - val_loss: 0.135 - valid-F1-score: 0.846
Epoch 4 - Loss: 0.081 - F1-score: 0.895 - val_loss: 0.153 - valid-F1-score: 0.768
Epoch 5 - Loss: 0.063 - F1-score: 0.927 - val_loss: 0.132 - valid-F1-score: 0.849


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at Twitter/twhin-bert-large and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 - Loss: 0.354 - F1-score: 0.310 - val_loss: 0.230 - valid-F1-score: 0.755
Epoch 2 - Loss: 0.157 - F1-score: 0.779 - val_loss: 0.158 - valid-F1-score: 0.839
Epoch 3 - Loss: 0.105 - F1-score: 0.854 - val_loss: 0.151 - valid-F1-score: 0.844
Epoch 4 - Loss: 0.077 - F1-score: 0.893 - val_loss: 0.150 - valid-F1-score: 0.855


In [6]:
import pandas as pd
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
wrong_predcit[0]

,text,different_emotions
0,всё так я тоже впечатлен поведением укр церкви в текущей ситуаци,"[surprise: predict=0, label=1]"
1,Да блин ! Ну Детка ты должна зажечь,"[anger: predict=1, label=0, joy: predict=0, label=1]"
2,чувствую новый год будет полнейшим дерьмом,"[disgust: predict=1, label=0]"
3,о боже мой даже не верится что прошел уже целый год с тех пор как ничего не изменилось,"[joy: predict=1, label=0]"
4,И дана улице просто пиздец Такой ветерчто мне кажется мою пятиэтажку унесет,"[fear: predict=0, label=1]"
5,Завтра к третьему уроку,"[joy: predict=0, label=1]"
6,Как вы со мной общаетесь ? На меня же смотреть противно,"[surprise: predict=0, label=1]"
7,Смотрю повтор речи Чернышенко это конечно ужас тербл я так говорил на инглише когда мне было 10 ! Олимпиада Мутко английский,"[disgust: predict=0, label=1, fear: predict=1, label=0]"
8,Черепаха клаустрофоб живёт в ужасе,"[fear: predict=1, label=0]"
9,Здоровый крепкий сон это когда ставишь будильник на повтор через каждые 5 минут и каждый раз успеваешь заснуть,"[joy: predict=1, label=0]"


In [7]:
wrong_predcit[1]

,text,different_emotions
0,Юна ты всё больше удивляешь меня своими достижениями,"[joy: predict=0, label=1]"
1,Людиотличная новость 47 школы закрыли на карантин краснокаменск,"[surprise: predict=1, label=0]"
2,Вот Рима коза ! не проверила тесты !,"[anger: predict=0, label=1, joy: predict=1, label=0, surprise: predict=1, label=0]"
3,Мда уж Дрожащими руками пишу бля радость,"[fear: predict=0, label=1, joy: predict=1, label=0]"
4,о боже Аля Киркоров радость,"[joy: predict=1, label=0]"
5,Теракт на вокзале в Волгограде По предварительным даным не мене 18 человек погибли Жесть,"[fear: predict=0, label=1, sadness: predict=0, label=1, surprise: predict=1, label=0]"
6,Пить одеколон ужасно,"[sadness: predict=1, label=0]"
7,Учимся 31 декабря провал этоненорма пиздец,"[anger: predict=0, label=1, sadness: predict=1, label=0]"
8,Мне приснился сегодня Курт Кобейн И я теперь как то ошарашена Он сказал мне такое над чем мне реально пришлось задуматься,"[surprise: predict=0, label=1]"
9,Воу круто,"[joy: predict=0, label=1]"


In [8]:
wrong_predcit[2]

,text,different_emotions
0,отвратительно когда люди просто без причины забывают старых друзей переключаясь на новых,"[anger: predict=0, label=1, sadness: predict=1, label=0]"
1,Тигруля,"[joy: predict=0, label=1]"
2,О Боже ж ты мой ! Я и не заметила что последний твит был 1000 ! Ну что ж смело перешагиваю эту цифру и 1001 твит !,"[joy: predict=1, label=0]"
3,вау !,"[joy: predict=0, label=1]"
4,Вот дерьмо !,"[disgust: predict=1, label=0, sadness: predict=0, label=1]"
5,ты мерзкийкак весь мир мерзкий,"[anger: predict=1, label=0]"
6,Бля убейте меня,"[anger: predict=0, label=1, fear: predict=1, label=0]"
7,Просочившаяся из под двери вонь быстро распространилась по всему зДаниию,"[disgust: predict=0, label=1, fear: predict=1, label=0]"
8,Мама наказала так что теперь не гулять не телефона ! Короче полный пиздец !,"[anger: predict=1, label=0]"
9,ДЕЛО Владельцы Nokia Lumia могут создавать папки,"[joy: predict=1, label=0]"


In [9]:
wrong_predcit[3]

,text,different_emotions
0,Надоело уже быть крошкой и маденьким солнышком Я не настолько младше тебя смущенный,"[disgust: predict=1, label=0]"
1,Планы могут меняться и к лучшему так ? Утомительные поиски приводят к цели рано или поздно,"[surprise: predict=1, label=0]"
2,кстативсе равно мы молодцы ! наша группа заняла второе место в олимпиаде пиздюки первые грусть,"[joy: predict=0, label=1, sadness: predict=1, label=0]"
3,люди такие добрые ! я просто поражаюсь реально добрые,"[joy: predict=1, label=0]"
4,ух даже не верится что я предыдущий твит не редактируя сразу в 140 символов уместила,"[joy: predict=0, label=1]"
5,Стойкое ощущение что кроме шлюхи ничего дельного из меня не выйдет шлюха пиздец конец безысходность тлен боль,"[fear: predict=0, label=1]"
6,не спала всю ночь:о теперь как зомби пиздец зомби апокалипсис недосып,"[anger: predict=1, label=0, sadness: predict=0, label=1]"
7,У меня одной появляется отвращение к человеку который пишет не грамотно ?,"[anger: predict=1, label=0, disgust: predict=0, label=1]"
8,ух ты что я нашла радость чуть меньше года назад написала заметку об одной компани и вот теперь я в ней работаю,"[joy: predict=0, label=1]"
9,Как же невозможно жить в этом мире лжи ненависти и боли грусть,"[disgust: predict=1, label=0]"


In [10]:
wrong_predcit[4]

,text,different_emotions
0,Мысль о смерти страшне смерти читаювзаимно,"[fear: predict=0, label=1]"
1,Не слушайте тех кто хорошо о вас отзывается Вы как были ничтожными кусками говна так ими и останетесь,"[disgust: predict=0, label=1]"
2,Эх сегодня прекрасная погода ! На улице 10 ! я в шоке но вобще круто радость иду себе наушники напялила и иду рэп слушаю радость незабываемое чувство,"[surprise: predict=0, label=1]"
3,Что бы ни происходило все хорошо Мудрость Дзен,"[joy: predict=0, label=1, sadness: predict=1, label=0]"
4,отвратный диджей в фокс кафе у меня ужасно болят уши от быдло музыки,"[anger: predict=0, label=1]"
5,Пиздец в хорошем смысле ! Голос,"[joy: predict=0, label=1, sadness: predict=1, label=0]"
6,В шоке от погоды !,"[anger: predict=0, label=1]"
7,Я вобще в шоке ! Как можно выпрашивать чтобы за тобой ухаживали и делали романтики ? радость,"[disgust: predict=0, label=1]"
8,Офигеть на улице как будто весна,"[joy: predict=0, label=1]"
9,Самое ужасноеэто выход на работу раньше времени честное слово я устала от всего этого,"[anger: predict=1, label=0]"


In [11]:
if best_models:
    best_model = min(best_models, key=lambda x: x['loss']) 
    print(f"\nUsing best model from fold {best_model['fold']} for prediction (loss: {best_model['loss']:.4f})")
    predict(best_model['model'], test_loader, test_df, 0.43)
else:
    print("\nNo models met the minimum loss criterion. Using the last trained model for prediction.")
    predict(model, test_loader, test_df, th)


No models met the minimum loss criterion. Using the last trained model for prediction.
